FRED data: 
- VIX (volatility index): Measures market fear/uncertainty. Spikes during crises.
- 10Y treasury: 
- 2Y treasury:
- Yield curve: (10Y - 2Y) Predicts economic cycles. Inversion often precedes recessions
- BAA corporate Bond: Captures perceived credit risk in corporate bonds. Widens during financial stress.
- CPI: Indicates inflationary environment. Influences policy and real returns.
- Unemployment rate: Reflects slack in the labour market. Lags the business cycle but still valuable.
- Fed Funds Rate: Measures monetary policy stance. Sharp changes can trigger regime shifts.
- GDP growth: measures real economic activity (however it is in quarterly not monthly so may have to forward fill to each corresponding month i.e. fill each missing month with the most recent known GDP value)
- Credit spread (BAA - GS10) (calculated)
- YoY Inflation = 12-month % change in CPI 

In [49]:
import pandas as pd
from datetime import datetime
from fredapi import Fred
from dotenv import load_dotenv
import os


In [50]:
load_dotenv()

fred_api_key = os.getenv("FRED_API_KEY")

f = Fred(api_key=fred_api_key) # Instantiate FRED with API key 


In [51]:
# Define series of interest
series = {
    'VIX': 'VIXCLS',
    '10Y': 'GS10',
    '2Y': 'GS2',
    'YieldCurve': 'T10Y2Y',
    'BAA': 'BAA', # Corporate bond yield (credit risk)
    'CPI': 'CPIAUCSL',
    'UnemploymentRate': 'UNRATE',
    'FedFundsRate': 'FEDFUNDS',
    'GDP_YoY': 'A191RL1Q225SBEA'
}

In [52]:
start_date = '1963-07-01'
end_date = '2025-05-31'


In [53]:
macro_data = {}
for name, code in series.items():
    macro_data[name] = f.get_series(code, observation_start=start_date, observation_end=end_date)


In [54]:
macro_df = pd.DataFrame(macro_data) # Convert each to a dataframe


In [55]:
macro_df['Date'] = macro_df.index # Add date as a column


In [56]:
# Compute derived columns
macro_df['CreditSpread'] = macro_df['BAA'] - macro_df['10Y']
macro_df['Inflation_YoY'] = macro_df['CPI'].pct_change(periods=12) * 100
macro_df['GDP_YoY'] = macro_df['GDP_YoY'].ffill()


C:\Users\User\AppData\Local\Temp\ipykernel_21872\1166655369.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  macro_df['Inflation_YoY'] = macro_df['CPI'].pct_change(periods=12) * 100


In [57]:
# Reset index and reorder
macro_df.reset_index(drop=True, inplace=True)
macro_df = macro_df[['Date', 'VIX', '2Y', '10Y', 'YieldCurve', 'CreditSpread',
                 'FedFundsRate', 'Inflation_YoY', 'UnemploymentRate', 'GDP_YoY']]


Now need to make it monthly in line with factor data

In [58]:
macro_df.set_index('Date', inplace=True)
monthly_macro = macro_df.resample('MS').mean()
monthly_macro.reset_index(inplace=True)


In [60]:
monthly_macro.to_csv('monthly_macro.csv', index=False)